In [1]:
!pip install --upgrade pyscreeze Pillow playsound opencv-python


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [2]:
import pyscreeze
import PIL
import cv2
import pyautogui
import pyautogui
import time
from datetime import datetime
import cv2
import subprocess
import time
from openai import Client

# this was needed for weird bug comparing string vs float for library versions
# https://stackoverflow.com/questions/76361049/how-to-fix-typeerror-not-supported-between-instances-of-str-and-int-wh
__PIL_TUPLE_VERSION = tuple(int(x) for x in PIL.__version__.split("."))
pyscreeze.PIL__version__ = __PIL_TUPLE_VERSION


## First try GPT-4 vision with an image

In [3]:
import base64
filepath = "screenshot_2023-11-08_19-53-02.png"
base64_image = base64.b64encode(open(filepath, "rb").read()).decode("utf-8")
base64_image

'...'

In [4]:
from openai import OpenAI

client = OpenAI()


def load_image_for_gpt(image_path):
    import base64
    base64_image = base64.b64encode(open(image_path, "rb").read()).decode("utf-8")
    return base64_image

# more infos on gpt-4-vision api in here: https://platform.openai.com/docs/guides/vision
def get_image_descriptions(base64_images):
    content_images = [{"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}} for base64_image in base64_images]
    response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "From the webcam screenshot describe my mood, if I am focus or looking around. Additionally, Grade the screenshot productivity between 0 and 5, and explain (5 means everything is for getting work done in my screen, while 0 is not, for example scrolling facebook or youtube)"}
            ] + content_images,
        }
    ],
    max_tokens=1000,
    )
    return response.choices[0].message.content

In [6]:
img_scrennshot = load_image_for_gpt("screenshot_2023-11-08_19-53-02.png")
img_webcam = load_image_for_gpt("webcam_2023-11-08_19-53-02.jpg")
get_image_descriptions([img_webcam, img_scrennshot])

"Based on the first image, you appear to be focused on something directly in front of you, likely a task on your computer screen. Your eyes are directed downwards towards the screen, indicating concentration, and your facial expression is neutral, which suggests a level of attention and involvement with what you're doing.\n\nRegarding the second screenshot, analyzing the productivity score, it's clear that you are working with a code editor and have a script or a code notebook open. The content shown includes programming scripts and functions, with references to using APIs, which is work-related for software development, data analysis, or machine learning tasks. As such, I would give this screenshot a productivity score of 5, as everything visible on the screen aligns with a work-related context and there are no signs of unrelated social media or entertainment platforms."

## Now try capturing the webcam


In [7]:
def capture_screenshots_and_webcam():
    # Open the webcam device
    cap = cv2.VideoCapture(0)  # '0' is usually the default value for the first webcam
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    # Get screen size
    screen_width, screen_height = pyautogui.size()

    
    # Generate a timestamp for the filename
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    
    # Take a screenshot with pyautogui
    screenshot_filename = f'screenshot_{timestamp}.png'
    screenshot = pyautogui.screenshot()
    # resize a bit before saving
    screenshot = screenshot.resize((int(screen_width/1.5), int(screen_height/1.5)), )
    screenshot.save(screenshot_filename, quality=50)
    print(f'Screenshot saved as {screenshot_filename}')
    
    # Capture frame-by-frame from the webcam
    ret, frame = cap.read()
    if ret:
        # Resize webcam image to match the screen size
        frame_resized = cv2.resize(frame, (screen_width, screen_height), interpolation=cv2.INTER_AREA)
        
        webcam_filename = f'webcam_{timestamp}.jpg'
        # compress imwrite 
        cv2.imwrite(webcam_filename, frame_resized, [int(cv2.IMWRITE_JPEG_QUALITY), 50])
        print(f'Webcam image saved as {webcam_filename}')
    else:
        raise Exception("Failed to capture image from webcam")
    return screenshot_filename, webcam_filename

capture_screenshots_and_webcam()

Screenshot saved as screenshot_2023-11-08_20-13-42.png
Webcam image saved as webcam_2023-11-08_20-13-42.jpg


('screenshot_2023-11-08_20-13-42.png', 'webcam_2023-11-08_20-13-42.jpg')

## Now create function to notify the user on the screen

In [12]:

def display_notification(title: str, subtitle: str, message: str, sound="Submarine"):
    # on mac only
    """
    Display a notification on macOS
    sound="Submarine" is the default sound - calm
    sound="Basso" for alert songs
    """
    applescript_command = f'''
    display notification "{message}" with title "{title}" subtitle "{subtitle}" sound name "{sound}"
    '''
    
    # Run the AppleScript command
    subprocess.run(["osascript", "-e", applescript_command])

# gentle
display_notification("Main Title", "Sub Title", "A congrats message", "Submarine")
time.sleep(5)
# alert
display_notification("Main Title", "Sub Title", "An alert message", "Basso")

## Instantiate assistant with the right parameters

In [14]:
function = {
    "type": "function",
    "function": {
      "name": "display_notification",
      "description": display_notification.__doc__.strip(),
      "parameters": {
        "type": "object",
        "properties": {
          "title": {"type": "string",},
          "subtitle": {"type": "string",},
          "message": {"type": "string",},
          "sound": {"type": "string",},
        },
        "required": ["title", "subtitle", "message", "sound"]
      }
    }
}
tools = [function]

In [15]:
import openai
instructions = """
You are my personal productivity assistant. You receive a description of my screenshot + my webcam - with grade around productivity.

If productivity is too low, send me a notification to warn me using `display_notification` function, explaining me the issue, and a small tips to get back to work. (short text)

If you see good productivity metric, send me a congrat , or eventually a small tips to keep going, or a poetic note
Also describe in the thread what I am doing in addition to sending notifications
- ALWAYS make a call to `display_message`
- The message is short (because notifications on apple notifications are short we cannot expand the message)
"""
name = "Personal Productivity assistant (API)"
productivity_assistant = openai.beta.assistants.create(
    instructions=instructions,
    name=name,
    tools=tools,
    model="gpt-4-1106-preview",
)


In [16]:
assistant_id = productivity_assistant.id
thread = openai.beta.threads.create()
thread.id

'thread_wwFIKseoNZcf3niqunthJaCl'

In [18]:
thread_id = thread.id

import json

def execute_required_action(tool_call):
    # making assumption we have a function 
    function = tool_call.function
    arguments = json.loads(function.arguments)
    function_name = function.name
    if function_name == "display_notification":        
        display_notification(**arguments)
    else:
        raise Exception(f"Unknown function {function_name}")

In [19]:
import time

from datetime import datetime

while True:
    # create pictures of both
    filename_screenshot, filename_webcam = capture_screenshots_and_webcam()

    screenshot_gpt_format = load_image_for_gpt(filename_screenshot)
    webcam_gpt_format = load_image_for_gpt(filename_webcam)    


    print("Sending images to GPT-4 Vision API...")
    images_descriptions = get_image_descriptions([screenshot_gpt_format, webcam_gpt_format])
    print(f"Images descriptions: {images_descriptions}")
    current_date_pretty = datetime.now().strftime('%H-%M-%S')
    query = f"""
Date: {current_date_pretty}
Images descriptions: {images_descriptions}
"""
    message = client.beta.threads.messages.create(thread_id=thread.id, role="user", content=query)
    

    current_time = datetime.now() # send a message every 20 seconds
    print(f"Sent message at {current_time}, launching the run...")
    run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=assistant_id)
    while True:
        run = client.beta.threads.runs.retrieve(run.id,thread_id=thread_id)
        if run.required_action:
            print(f"Required action: for the run {run.id}")

            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []
            for tool_call in tool_calls:
                print(f"Tool call: {tool_call}")
                output = execute_required_action(tool_call) # might not have any outputs if this is just a notification
                tool_outputs.append({"tool_call_id": tool_call.id, "output": str(output)})

            run = openai.beta.threads.runs.submit_tool_outputs(thread_id=thread_id,run_id=run.id,tool_outputs=tool_outputs)
            print(f"Required action completed for the run {run.id}")
            print(run.completed_at)
            
        if run.completed_at:
            messages = client.beta.threads.messages.list(thread_id=thread_id)
            last_message = messages.data[0]
            last_content = last_message.content[0]
            print(last_content.text.value)
            break
    # compare with current_time and check if > 30 seconds
    delta_time = datetime.now() - current_time
    if delta_time.seconds < 30:
        print(f"Waiting for remaining time {30 - delta_time.seconds} seconds")
        time.sleep(30 - delta_time.seconds)


Screenshot saved as screenshot_2023-11-08_20-22-37.png
Webcam image saved as webcam_2023-11-08_20-22-37.jpg
Sending images to GPT-4 Vision API...
Images descriptions: Based on the images provided:

Mood and Focus: The individual appears to be directed towards the camera or screen with a neutral expression on their face. There aren't obvious signs of strong emotion, suggesting a calm or focused demeanor. It is not possible to unequivocally determine their mood from a static image, but the relaxed facial features might imply a level of concentration or attentiveness to the task at hand.

Productivity Grade: In the screenshot of the computer screen, the individual appears to be working within a programming environment, specifically VS Code, with a Jupyter notebook open. The content on the screen includes code and terminal commands which are indicative of a productive work session. Given that the applications and windows in the screenshot are related to programming and development tasks, I

In [58]:
print(query)


Date: 19-46-48
Images descriptions: The first image is a screenshot of a computer screen showing a Jupyter Notebook, which is an open-source web application that allows you to create and share documents that contain live code, equations, visualizations, and narrative text. The code appears to be written in Python and includes comments relating to version control with Git, a version control system for tracking changes in computer files and coordinating work on those files among multiple people. This is a tool used by software developers for source code management and can play a significant role in increasing productivity and collaboration in software development.

The second image is a webcam snapshot of a person who appears to be looking intently at the screen, potentially making them feel more focused or self-aware while working. This could be a setup where the individual monitors themselves to stay on task, stay attentive during online meetings, or analyze their focus levels at work

In [19]:
print(last_content.text.value)

It looks like you are staying productive and working on debugging, which is great to see. Monitoring your progress and maintaining focus during development is key to creating effective code.

If you're struggling to resolve a bug or find a way through a complex piece of logic, sometimes a short break or a change in perspective can help. Step back for a moment, take a deep breath, and revisit the problem. Often, this can lead to a fresh idea or solution that wasn't apparent during continuous work.

Keep on tackling those challenges one step at a time. Remember, every problem solved is progress made, and I'm here to support your productivity journey!


In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id)

In [ ]:
run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
run.completed_at

In [ ]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

for message in messages.data:
    print(message.content[0].text.value)